# A class for accessing data

This class could be defined in another place but I feel it will be good to allocate it in this place for now

In [1]:
import os
import csv
import sys
import re
import heapq
from collections import defaultdict
from operator import itemgetter
from surprise import Dataset, Reader, SVD, accuracy, KNNBasic

In [2]:
class MovieLens:

    movieID_to_name = {}
    name_to_movieID = {}
    ratingsPath = 'C:\\Users\\joseh\\Anaconda Projects\\Personal studies\\RecSys Course\\ml-latest-small\\ratings ut.csv'
    moviesPath = 'C:\\Users\\joseh\\Anaconda Projects\\Personal studies\\RecSys Course\\ml-latest-small\\movies.csv'
    
    def loadMovieLensLatestSmall(self):

        # Look for files relative to the directory we are running from
        os.chdir(os.path.dirname(sys.argv[0]))

        ratingsDataset = 0
        self.movieID_to_name = {}
        self.name_to_movieID = {}

        reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)

        ratingsDataset = Dataset.load_from_file(self.ratingsPath, reader=reader)

        with open(self.moviesPath, newline='', encoding='ISO-8859-1') as csvfile:
                movieReader = csv.reader(csvfile)
                next(movieReader)  #Skip header line
                for row in movieReader:
                    movieID = int(row[0])
                    movieName = row[1]
                    self.movieID_to_name[movieID] = movieName
                    self.name_to_movieID[movieName] = movieID

        return ratingsDataset
    
    def getMovieName(self, movieID):
        if movieID in self.movieID_to_name:
            return self.movieID_to_name[movieID]
        else:
            return ""
        
    def getMovieID(self, movieName):
        if movieName in self.name_to_movieID:
            return self.name_to_movieID[movieName]
        else:
            return 0

# Data, parameters and others

Reading data

In [3]:
ml = MovieLens()
data = ml.loadMovieLensLatestSmall()

Split data

In [4]:
trainSet = data.build_full_trainset()

# Model user-based

In [5]:
model = KNNBasic(sim_options={'name': 'cosine','user_based': False})
model.fit(trainSet)
simsMatrix = model.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [16]:
testSubject = '672'
k = 50

In [17]:
testUserInnerID = trainSet.to_inner_uid(testSubject)

In [18]:
# Get the top K items we rated
testUserRatings = trainSet.ur[testUserInnerID]
#kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])
kNeighbors = []

for rating in testUserRatings:
    if rating[1]> 4.0:
        kNeighbors.append(rating)

In [19]:
# Get similar items to stuff we liked (weighted by rating)
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    similarityRow = simsMatrix[itemID]
    for innerID, score in enumerate(similarityRow):
        candidates[innerID] += score * (rating / 5.0)

In [20]:
# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1

## Generate top-N

In [21]:
# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break

Ipcress File, The (1965) 6.500000000000001
Deterrence (1999) 6.500000000000001
8 Â½ Women (a.k.a. 8 1/2 Women) (a.k.a. Eight and a Half Women) (1999) 6.500000000000001
Criminal Lovers (1999) 6.500000000000001
AimÃ©e & Jaguar (1999) 6.500000000000001
Amazon Women on the Moon (1987) 6.500000000000001
Barfly (1987) 6.500000000000001
King Is Alive, The (2000) 6.500000000000001
Jump Tomorrow (2001) 6.500000000000001
Innocence (2000) 6.500000000000001
Medium Cool (1969) 6.500000000000001
